In [ ]:
import requests
import json
from datetime import datetime

In [ ]:
class GitHubRepository:
    def __init__(self, name, owner,  description, homepage, license, forks, watchers, date_of_collection):
        self.name = name
        self.owner = owner
        self.description = description
        self.homepage = homepage
        self.license = license  # License could be another class with its own attributes
        self.forks = forks
        self.watchers = watchers
        self.date_of_collection = date_of_collection

    @staticmethod
    def parse(dict):
      name = dict['name'] if 'name' in dict else None
      owner = dict['owner']['login'] if 'owner' in dict and 'login' in dict['owner'] else None
      description = dict['description'] if 'description' in dict else None
      homepage = dict['homepage'] if 'homepage' in dict else None
      forks = dict['forks'] if 'forks' in dict else None
      watchers = dict['watchers'] if 'watchers' in dict else None
      license = dict['license']['name'] if 'license' in dict and dict['license'] != None and 'name' in dict['license'] else None

      #get current time in UTC
      1971

      time = datetime.now()
      formatted_time = time.strftime("%Y-%m-%dT%H:%M:%SZ")

      return GitHubRepository(name, owner, description, homepage, license, forks, watchers, formatted_time)

    def __str__(self):
        return f"{self.owner}/{self.name}: license = [{self.license}] homepage = [{self.homepage}] forks = ({self.forks}) watchers = ({self.watchers}) date of collection: {self.date_of_collection}"

In [ ]:
class PullRequest:
  def __init__(self,user, title, number, body, state, created_at, closed_at):
    self.user = user
    self.title = title
    self.number = number
    self.body = body
    self.state = state
    self.created_at = created_at
    self.closed_at = closed_at

  @staticmethod
  def parse(dict):
    title = dict['title'] if 'title' in dict else None
    user = dict['user']['login'] if 'user' in dict and 'login' in dict['user'] else None
    number = dict['number'] if 'number' in dict else None
    body = dict['body'] if 'body' in dict else None
    state = dict['state'] if 'state' in dict else None
    created_at = dict['created_at'] if 'created_at' in dict else None
    closed_at = dict['closed_at'] if 'closed_at' in dict else None

    return PullRequest(user,title,number,body,state,created_at,closed_at)

  def __str__(self):
    return f"{self.user}/{self.title}: ({self.state}) ({self.number}) ({self.created_at}) ({self.closed_at})"

In [ ]:
class PRStatus:
  def __init__(self,number,commits,additions,deletions,changed_files):
    self.number = number
    self.commits = commits
    self.additions = additions
    self.deletions = deletions
    self.changed_files = changed_files

  @staticmethod
  def parse(dict):
    number = dict['number'] if 'number' in dict else None
    commits = dict['commits'] if 'commits' in dict else None
    additions = dict['additions'] if 'additions' in dict else None
    deletions = dict['deletions'] if 'deletions' in dict else None
    changed_files = dict['changed_files'] if 'changed_files' in dict else None

    return PRStatus(number,commits,additions,deletions,changed_files)

  def __str__(self):
    return f"PullRequest #{self.number}: commits = ({self.commits}) additions = ({self.additions}) deletions = ({self.deletions}) file changes = ({self.changed_files})"

In [ ]:
class UserInfo:
  def __init__(self, name, no_of_repos, followers, following):
    self.name = name
    self.no_of_repos = no_of_repos
    self.followers = followers
    self.following = following

  @staticmethod
  def parse(dict):
    name = dict['name'] if 'name' in dict else None
    no_of_repos = dict['public_repos'] if 'public_repos' in dict else None
    followers = dict['followers'] if 'followers' in dict else None
    following = dict['following'] if 'following' in dict else None

    return UserInfo(name, no_of_repos, followers, following)

  def set_contributions(self,contributions):
    self.contributions = contributions



  def __str__(self):
    return f'{self.name}: repos = {self.no_of_repos}, followers = {self.followers}, following = {self.following}, contributions in last year = {self.contributions}'

In [ ]:
# Get Repository

owner = "debsourav33"
repo_name = "Leetcode-Solutions"

def get_repository(owner, repo) -> GitHubRepository :

  url = f"https://api.github.com/repos/{owner}/{repo}"
  response = requests.get(url)

  if response.status_code != 200:
    return None

  resp_dict = response.json()
  pretty_response = json.dumps(resp_dict, indent = 4)
  #print(pretty_response)

  return GitHubRepository.parse(resp_dict)

github_repo = get_repository('ramnes', 'notion-sdk-py')
print(github_repo)

In [ ]:
# Get Pull Request

owner = "jabref"
repo = "jabref"

def getPullRequests(owner, repo):
  url = f"https://api.github.com/repos/{owner}/{repo}/pulls"
  print(url)
  response = requests.get(url)

  if response.status_code != 200:
      print(f"Status code: {response.status_code}")
      return None
  else:
    resp_list = response.json()
    #print(len(resp_list))
    ret = []
    for item in resp_list:
      pr= PullRequest.parse(item)
      ret.append(pr)
      #print(pr)
    return ret


def getPullRequestInfo(owner, repo, number):
  url = f"https://api.github.com/repos/{owner}/{repo}/pulls/{number}"
  response = requests.get(url)

  if response.status_code != 200:
      print(f"Failed to retrieve pull requests. Status code: {response.status_code}")
      return None
  else:
    resp_dict = response.json()
    #print(type(resp_dict))
    #print(resp_dict)
    pr_info = PRStatus.parse(resp_dict)
    #print(pr_info)
    return pr_info



pull_requests = getPullRequests(owner,repo)
limit = 5 #so that we don't reach api call limit

for i in range(min(limit,len(pull_requests))):
  pr = pull_requests[i]
  if pr.number != None:
    pr_info = getPullRequestInfo(owner, repo, pr.number)
    print(pr_info)


In [ ]:
# Get User Info

user = "jabref"


def getUserInfo(user):
  url = f'https://api.github.com/users/{user}'

  user_info = None
  response = requests.get(url)

  if response.status_code != 200:
    print(f"Status code for {url}: {response.status_code}")
    return None
  else:
    resp_dict = response.json()
    user_info = UserInfo.parse(resp_dict)
    #print(json.dumps(resp_dict,indent = 4))

  #contributions
  url = f"https://api.github.com/users/{user}/events"
  response = requests.get(url)

  if response.status_code != 200:
    print(f"Status code for {url}: {response.status_code}")
  else:
    events = response.json()
    contributions = 0

    for event in events:
      if event.get('type','null') == 'PushEvent':
        #print(json.dumps(event, indent=4))

        #check if the contribution was made this year
        created_at = event.get('created_at','')
        if created_at != '':
          created_year = datetime.strptime(created_at, '%Y-%m-%dT%H:%M:%SZ').year
          curr_year = datetime.now().year

          if curr_year == created_year:
            contributions += 1


    if user_info != None:
      user_info.set_contributions(contributions)

    return user_info

user_info = getUserInfo(user)
print(user_info)

In [ ]:
# import the unittest module
import unittest
from datetime import datetime

# create a test class that inherits from unittest.TestCase
class TestGitHubRepository(unittest.TestCase):

    # use the setUp method to perform any setup actions before each test
    def setUp(self):
        # create a sample dictionary to use as input for the parse method
        self.sample_dict = {
            "name": "python-testing",
            "owner": {
                "login": "realpython"
            },
            "description": "A tutorial on testing in Python",
            "homepage": "https://realpython.com/python-testing/",
            "license": {
                "name": "MIT License"
            },
            "forks": 42,
            "watchers": 100
        }

    # write test methods that start with the word test
    def test_init(self):
        # test the __init__ method of the class
        repo = GitHubRepository("python-testing", "realpython", "A tutorial on testing in Python", "https://realpython.com/python-testing/", "MIT License", 42, 100, "2023-11-21T13:29:55Z")
        # use assert methods to check the expected output or behavior of the code under test
        self.assertEqual(repo.name, "python-testing")
        self.assertEqual(repo.owner, "realpython")
        self.assertEqual(repo.description, "A tutorial on testing in Python")
        self.assertEqual(repo.homepage, "https://realpython.com/python-testing/")
        self.assertEqual(repo.license, "MIT License")
        self.assertEqual(repo.forks, 42)
        self.assertEqual(repo.watchers, 100)
        self.assertEqual(repo.date_of_collection, "2023-11-21T13:29:55Z")

    def test_parse(self):
        # test the parse method of the class
        repo = GitHubRepository.parse(self.sample_dict)
        # check that the attributes are assigned correctly from the dictionary
        self.assertEqual(repo.name, "python-testing")
        self.assertEqual(repo.owner, "realpython")
        self.assertEqual(repo.description, "A tutorial on testing in Python")
        self.assertEqual(repo.homepage, "https://realpython.com/python-testing/")
        self.assertEqual(repo.license, "MIT License")
        self.assertEqual(repo.forks, 42)
        self.assertEqual(repo.watchers, 100)
        # check that the date_of_collection is the current time in UTC
        time = datetime.now()
        formatted_time = time.strftime("%Y-%m-%dT%H:%M:%SZ")
        self.assertEqual(repo.date_of_collection, formatted_time)

    def test_str(self):
        # test the __str__ method of the class
        repo = GitHubRepository.parse(self.sample_dict)
        # check that the string representation is formatted correctly
        expected_str = "realpython/python-testing: license = [MIT License] homepage = [https://realpython.com/python-testing/] forks = (42) watchers = (100) date of collection: " + repo.date_of_collection
        self.assertEqual(str(repo), expected_str)

    # use the tearDown method to perform any cleanup actions after each test
    def tearDown(self):
        # delete the sample dictionary
        del self.sample_dict

# create a test class that inherits from unittest.TestCase
class TestPullRequest(unittest.TestCase):
    # use the setUp method to perform any setup actions before each test
    def setUp(self):
        # create a sample dictionary to use as input for the parse method
        self.sample_dict = {
            "title": "Add unit tests",
            "user": {
                "login": "realpython"
            },
            "number": 42,
            "body": "This pull request adds some unit tests for the GitHubRepository and PullRequest classes.",
            "state": "open",
            "created_at": "2023-11-21T13:29:55Z",
            "closed_at": None
        }

    # write test methods that start with the word test
    def test_init(self):
        # test the __init__ method of the class
        pr = PullRequest("realpython", "Add unit tests", 42, "This pull request adds some unit tests for the GitHubRepository and PullRequest classes.", "open", "2023-11-21T13:29:55Z", None)
        # use assert methods to check the expected output or behavior of the code under test
        self.assertEqual(pr.user, "realpython")
        self.assertEqual(pr.title, "Add unit tests")
        self.assertEqual(pr.number, 42)
        self.assertEqual(pr.body, "This pull request adds some unit tests for the GitHubRepository and PullRequest classes.")
        self.assertEqual(pr.state, "open")
        self.assertEqual(pr.created_at, "2023-11-21T13:29:55Z")
        self.assertEqual(pr.closed_at, None)

    def test_parse(self):
        # test the parse method of the class
        pr = PullRequest.parse(self.sample_dict)
        # check that the attributes are assigned correctly from the dictionary
        self.assertEqual(pr.user, "realpython")
        self.assertEqual(pr.title, "Add unit tests")
        self.assertEqual(pr.number, 42)
        self.assertEqual(pr.body, "This pull request adds some unit tests for the GitHubRepository and PullRequest classes.")
        self.assertEqual(pr.state, "open")
        self.assertEqual(pr.created_at, "2023-11-21T13:29:55Z")
        self.assertEqual(pr.closed_at, None)

    def test_str(self):
        # test the __str__ method of the class
        pr = PullRequest.parse(self.sample_dict)
        # check that the string representation is formatted correctly
        expected_str = "realpython/Add unit tests: (open) (42) (2023-11-21T13:29:55Z) (None)"
        self.assertEqual(str(pr), expected_str)

    # use the tearDown method to perform any cleanup actions after each test
    def tearDown(self):
        # delete the sample dictionary
        del self.sample_dict

# create a test class that inherits from unittest.TestCase
class TestPRStatus(unittest.TestCase):

    # use the setUp method to perform any setup actions before each test
    def setUp(self):
        # create a sample dictionary to use as input for the parse method
        self.sample_dict = {
            "number": 42,
            "commits": 3,
            "additions": 100,
            "deletions": 50,
            "changed_files": 5
        }

    # write test methods that start with the word test
    def test_init(self):
        # test the __init__ method of the class
        pr_status = PRStatus(42, 3, 100, 50, 5)
        # use assert methods to check the expected output or behavior of the code under test
        self.assertEqual(pr_status.number, 42)
        self.assertEqual(pr_status.commits, 3)
        self.assertEqual(pr_status.additions, 100)
        self.assertEqual(pr_status.deletions, 50)
        self.assertEqual(pr_status.changed_files, 5)

    def test_parse(self):
        # test the parse method of the class
        pr_status = PRStatus.parse(self.sample_dict)
        # check that the attributes are assigned correctly from the dictionary
        self.assertEqual(pr_status.number, 42)
        self.assertEqual(pr_status.commits, 3)
        self.assertEqual(pr_status.additions, 100)
        self.assertEqual(pr_status.deletions, 50)
        self.assertEqual(pr_status.changed_files, 5)

    def test_str(self):
        # test the __str__ method of the class
        pr_status = PRStatus.parse(self.sample_dict)
        # check that the string representation is formatted correctly
        expected_str = "PullRequest #42: commits = (3) additions = (100) deletions = (50) file changes = (5)"
        self.assertEqual(str(pr_status), expected_str)

    # use the tearDown method to perform any cleanup actions after each test
    def tearDown(self):
        # delete the sample dictionary
        del self.sample_dict

# create a test class that inherits from unittest.TestCase
class TestUserInfo(unittest.TestCase):
    # use the setUp method to perform any setup actions before each test
    def setUp(self):
        # create a sample dictionary to use as input for the parse method
        self.sample_dict = {
            "name": "realpython",
            "public_repos": 10,
            "followers": 1000,
            "following": 50
        }

    # write test methods that start with the word test
    def test_init(self):
        # test the __init__ method of the class
        user_info = UserInfo("realpython", 10, 1000, 50)
        # use assert methods to check the expected output or behavior of the code under test
        self.assertEqual(user_info.name, "realpython")
        self.assertEqual(user_info.no_of_repos, 10)
        self.assertEqual(user_info.followers, 1000)
        self.assertEqual(user_info.following, 50)

    def test_parse(self):
        # test the parse method of the class
        user_info = UserInfo.parse(self.sample_dict)
        # check that the attributes are assigned correctly from the dictionary
        self.assertEqual(user_info.name, "realpython")
        self.assertEqual(user_info.no_of_repos, 10)
        self.assertEqual(user_info.followers, 1000)
        self.assertEqual(user_info.following, 50)

    def test_set_contributions(self):
        # test the set_contributions method of the class
        user_info = UserInfo.parse(self.sample_dict)
        # set the contributions attribute to a sample value
        user_info.set_contributions(500)
        # check that the attribute is updated correctly
        self.assertEqual(user_info.contributions, 500)

    def test_str(self):
        # test the __str__ method of the class
        user_info = UserInfo.parse(self.sample_dict)
        # set the contributions attribute to a sample value
        user_info.set_contributions(500)
        # check that the string representation is formatted correctly
        expected_str = "realpython: repos = 10, followers = 1000, following = 50, contributions in last year = 500"
        self.assertEqual(str(user_info), expected_str)

    # use the tearDown method to perform any cleanup actions after each test
    def tearDown(self):
        # delete the sample dictionary
        del self.sample_dict

# run the unit test using the unittest.main function with some arguments to avoid errors
unittest.main(argv=['first-arg-is-ignored'], exit=False)
